In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize

## Задача:

Задача имеет следующий вид:
$$
\begin{aligned}
    & \text{minimize}    && \sum_{t = 1}^{T}F(p_{eng}(t)) && \\
    & \text{subject to}  && p_{req}(t) = p_{eng}(t) + p_{mg}(t) - p_{br}(t),\quad t = 1,\ldots,T\\
    &                    && E(t + 1) = E(t) - p_{mg}(t) - \eta|p_{mg}(t)|\\
    &                    && 0 \leqslant E(t) \leqslant E_{batt}^{max}\\
    &                    && E(1) = E(T + 1) \\
    &                    && 0 \leqslant p_{eng}(t) \leqslant P_{eng}^{max}\\
    &                    && P_{mg}^{min} \leqslant p_{mg}(t) \leqslant P_{mg}^{max}\\
    &                    && 0 \leqslant p_{br}(t),\\
\end{aligned}
$$
Чтобы упростить задачу, уиз ограничения с модулем уберем данное слагаемое, то есть будем считать что система идеальна и энергия не рассеивается в окружающее пространство просто так. Тогда задача принимает вид:
$$
\begin{aligned}
    & \text{minimize}    && \sum_{t = 1}^{T}F(p_{eng}(t)) \\
    & \text{subject to}  && p_{req}(t) = p_{eng}(t) + p_{mg}(t) - p_{br}(t),\quad t = 1,\ldots,T \\
    &                    && E(t + 1) = E(t) - p_{mg}(t) \\
    &                    && 0 \leqslant E(t) \leqslant E_{batt}^{max} \\
    &                    && E(1) = E(T + 1) \\
    &                    && 0 \leqslant p_{eng}(t) \leqslant P_{eng}^{max} \\
    &                    && P_{mg}^{min} \leqslant p_{mg}(t) \leqslant P_{mg}^{max} \\
    &                    && 0 \leqslant p_{br}(t), \\
\end{aligned}
$$
Введем переменную $x = (p_{eng}(1), \ldots, p_{eng}(T), p_{mg}(1),\ldots,p_{mg}(T),p_{br}(1),\ldots,p_{br}(T),E(1),\ldots,E(T+1))$, $\dim x = 4T+1$, зададим $f_0(x)=\sum_{i = 0}^{T-1}F(x_i)$.
$$
\begin{aligned}
    & \text{minimize}    && f_0(x) \\
    & \text{subject to}  && p_{req}(t) = x_{i}(t) + x_{i+T}(t) - x_{i+2T}(t),\quad i = 0,\ldots,T-1 \\
    &                    && x_{j + 1} = x_j - x_{j-2T},\quad i = 3T,\ldots,4T-1 \\
    &                    && 0 \leqslant x_k \leqslant E_{batt}^{max},\quad k = 3T,\ldots,4T \\
    &                    && x_{3T} = x_{4T} \\
    &                    && 0 \leqslant x_l \leqslant P_{eng}^{max},\quad l = 0,\ldots,T-1 \\
    &                    && P_{mg}^{min} \leqslant x_{m} \leqslant P_{mg}^{max},\quad m = T,\ldots,2T-1 \\
    &                    && 0 \leqslant x_{r},\quad r = 2T,\ldots,3T-1 \\
\end{aligned}
$$
В матричной форме:
$$
\begin{aligned}
    & \text{minimize}    && f_0(x) \\
    & \text{subject to}  && Hx \preceq g \\
    &                    && Ax = b \\
\end{aligned}
$$

# Метод внутренней точки: 
## Поиск минимума с использованием логарифмической барьерной функции и метода Ньютона с ограничениями типа равенств. 

In [3]:
# Заданные параметры задачи
T = 5             # общее число точек фиксированного времени
P_mg_max = 0.5    # верхняя грань для p_mg
P_mg_min = -0.5   #  нижняя грань для p_mg
P_eng_max = 0.8   # верхняя грань для p_eng
E_bat_max = 1     # верхняя грань для E

p_req = np.array([0.375, 0.96, 0.74, 0.6, 0.17])   # требуемые значения p_req мощности при t = 1...T

In [4]:
# Матрица ограничений-равенств
str_1 = np.zeros(4*T + 1)
str_1[0], str_1[T], str_1[2*T] = 1, 1, -1
str_2 = np.zeros(4*T + 1)
str_2[T], str_2[3*T], str_2[3*T + 1] = 1, -1, 1
str_3 = np.zeros(4*T + 1)
str_3[3*T], str_3[-1] = 1, -1

A_mtx = np.zeros(shape=(2*T+1, 4*T+1))
for i in range(T):
    A_mtx[i] = np.roll(str_1, i)
    A_mtx[i+T] = np.roll(str_2, i)
A_mtx[-1] = str_3

# Правая часть ограничений-равенств
b = np.zeros(2*T + 1)
b[:T] = p_req

# Матрица ограничений-неравенств
H = np.zeros(shape=(7*T+2,4*T+1))
H[:(4*T+1),:] = np.diag(-np.ones(4*T+1))
H[(4*T+1):(6*T+1),:2*T] = np.diag(np.ones(2*T))
H[(6*T+1):, 3*T:] = np.diag(np.ones(T+1))

# Правая часть ограничений-неравенств
g = np.zeros(7*T+2)
g[T:2*T] = -P_mg_min*np.ones(T)
g[(4*T+1):(5*T+1)] = P_eng_max*np.ones(T)
g[(5*T+1):(6*T+1)] = P_mg_max*np.ones(T)
g[(6*T+1):] = E_bat_max*np.ones(T+1)

# Внутренняя точка допустимой области
x0 = np.array([0.5, 0.7, 0.7, 0.5, 0.65, -0.1, 0.3, 0.1, 0.15, -0.45, 0.025, 0.04, 0.06, 0.05, 0.03, 0.5, 0.6, 0.3, 0.2, 0.05, 0.5])

Воспользуемся барьерным методом решения задачи с ограничениями: к исходной функции прибавим логарифмическую барьерную:
$$
f(x) = f_0(x) - r\sum_{i = 0}^m \log(-g_i(x)),
$$
где $g_i(x)$ - линейные ограничения-неравенства в задаче. Таким образом, внутри допустимой области добавка будет малой, тогда как при приближении к границе области она будет неограниченно возрастать.

In [5]:
r = 1.   # коэф-т барьерного метода решения задачи с ограничениями

# Исходная функция F(x) = sum(x_i*log(x_i + 1))
F = lambda x: x * np.log(x + 1)
gradF = lambda x: np.log(x + 1) + x / (x + 1)
hessF = lambda x: np.diag((x + 2) / ((x + 1) **2))

def f(x):
    res = sum(F(x[:T]))                     # исходная функция
    res += (r * sum(-np.log(-(H @ x-g))))   # логарифмический барьер
    return res

# Градиент целевой функции
def gradf(x):
    res = np.zeros(4*T + 1)
    res[:T] = gradF(x[0:T])        # градиент исходной функции
    
    tmp = (H @ x-g)
    for i in range(H.shape[0]):
#         if tmp[i] == 0:
#             print("error")
        res += (r * (-H[i,:].T) / tmp[i])  # подсчет градиента барьерной части
    return res

# Гессиан целевой функции
def hessf(x):
    res = np.zeros(shape=(4*T+1, 4*T+1))
    res[:T,:T] += hessF(x[0:T])    # гессиан исходной функции
    tmp = (H @ x-g)
    for i in range(H.shape[0]):        # подсчет гессиана барьерной функции
        tmp_ = np.eye(H.shape[1])
        for j in range(H.shape[1]):
            for k in range(H.shape[1]):
                tmp_[j][k] = H[i, j] * H[i, k]
#                 if tmp[i] == 0:
#                     print("error")
        res += (r * tmp_ / (tmp[i] ** 2))
    return res

Усовершенстванный метод Ньютона для задачи с ограничениями типа равенств, выбор шага по способу Армихо, и кроме того, на каждом шаге осуществляетяс проверка того, что новая точка принадлежит допустимой области, если это не так, то шаг снова уменьшается.

In [6]:
# Проверка того, что точка лежит в допустимой области ограничений
def check(x_next):
    flag = True
    tmp = H @ x_next-g
    for i in range(len(tmp)):
        if tmp[i] >= 0:
            flag = False
    if np.abs(np.sum(A_mtx @ x_next - b)) > 1e-6:
#         print(np.sum(A_mtx @ x_next - b))
        flag = False
    return flag

# Метод Ньютона для решения задачи с ограничениями типа равенств
def modified_newton(f, gradf, hessf, x0, A, b, eps=1e-6):
    x = x0.copy()
    n = x.shape[0]
    size = n + A.shape[0]
    while True:
        B = np.zeros(shape=(size,size))
        B[:n,:n] = hessf(x)
        B[:n,n:] = A.T
        B[n:,:n] = A
        
        c = np.zeros(size)
        c[:n] = -gradf(x)
        step = np.linalg.solve(B, c)[:n]
        
        dec = np.sqrt(step.T @ hessf(x) @ step)
        if (dec ** 2) < 2 * eps:
            break
            
        alpha = 1
        beta = 0.1
        rho = 0.1
        x_next = x + alpha * step
        while not check(x_next) or f(x_next) > f(x) + beta * alpha * np.array(gradf(x)).dot(step):
            alpha *= rho
            x_next = x + alpha * step
        
        x = x_next
    return x

Решаем задачу барьерным способом: на каждой итерации уменьшаем значение $r$ и ищем следующую точку с помощью метода Ньютона для задачи с ограничениями-равенствами.

In [7]:
m = A_mtx.shape[0]
r = 1
rho = 0.1
num_iter = 0
x = x0.copy()
# print(x0, sum(F(x0[:T])), f(x0))
while True:
#     print("start_iteration:")
    x_next = modified_newton(f, gradf, hessf, x, A_mtx, b, 1e-6)
#     print("x_next:", x_next, sum(F(x_next[:T])), f(x_next))
    x = x_next
    if m * r < 1e-6:
        break
    r *= rho   
    num_iter += 1

In [8]:
print("Начальная допустимая точка x0 :", x0)
print("Значение функции F(x0) :", sum(F(x0[:T])), "\n")
print("Найденное решение x* :", x)
print("Значение функции F* :\t", sum(F(x[:T])))

Начальная допустимая точка x0 : [ 0.5    0.7    0.7    0.5    0.65  -0.1    0.3    0.1    0.15  -0.45
  0.025  0.04   0.06   0.05   0.03   0.5    0.6    0.3    0.2    0.05
  0.5  ]
Значение функции F(x0) : 1.4738485967383208 

Найденное решение x* : [ 5.68980020e-01  5.69072886e-01  5.69026284e-01  5.69000774e-01
  5.68920122e-01 -1.93980007e-01  3.90927149e-01  1.70973729e-01
  3.09992390e-02 -3.98920110e-01  1.25828054e-08  3.48567417e-08
  1.38380314e-08  1.28114905e-08  1.23665301e-08  6.23224872e-01
  8.17204879e-01  4.26277731e-01  2.55304001e-01  2.24304762e-01
  6.23224872e-01]
Значение функции F* :	 1.2814975347876951


In [ ]:
# print(x)
# print(sum(F(x[:T])))
# print(x[0:T] + x[T:2*T] - x[2*T:3*T])
# print(x[3*T+1:]-x[3*T:-1]+x[T:2*T])
# print(x[0:T])
# print(x[T:2*T])
# print(x[2*T:3*T])
# print(x[3*T:])

# Задача линейного программирования.
Возьмем в качестве функции $F$ линейную функцию, решим задачу тем же способом, и сравним полученный ответ с решением пакетного модуля cvxopt.

In [533]:
from cvxopt import matrix, solvers

In [557]:
c = np.zeros(4*T + 1)
c[:T] = np.array([1.5, 1.4, 1.8, 1.2, 2.6])

F = lambda x: sum(x * c)
gradF = lambda x: c
hessF = lambda x: np.zeros(4*T+1)

r = 1.

def f(x):
    res = F(x)                          # исходная функция
    res += (r * sum(-np.log(-(H @ x-g))))   # логарифмический барьер
    return res

# Градиент целевой функции
def gradf(x):
    res = np.zeros(4*T + 1)
    res += gradF(x)
#     res[:T] = gradF(x[0:T])        # градиент исходной функции
    
    tmp = (H @ x-g)
    for i in range(H.shape[0]):
#         if tmp[i] == 0:
#             print("error")
        res += (r * (-H[i,:].T) / tmp[i])  # подсчет градиента барьерной части
    return res

# Гессиан целевой функции
def hessf(x):
    res = np.zeros(shape=(4*T+1, 4*T+1))
#     res[:T,:T] += t * hessF(x[0:T])    # гессиан исходной функции
    tmp = (H @ x-g)
    for i in range(H.shape[0]):        # подсчет гессиана барьерной функции
        tmp_ = np.eye(H.shape[1])
        for j in range(H.shape[1]):
            for k in range(H.shape[1]):
                tmp_[j][k] = H[i, j] * H[i, k]
#                 if tmp[i] == 0:
#                     print("error")
        res += (r * tmp_ / (tmp[i] ** 2))
    return res

In [558]:
m = A_mtx.shape[0]
r = 1
rho = 0.1
num_iter = 0
x = x0.copy()
# print(x0, sum(F(x0[:T])), f(x0))
while True:
#     print("start_iteration:")
    x_next = modified_newton(f, gradf, hessf, x, A_mtx, b, 1e-6)
#     print("x_next:", x_next, sum(F(x_next[:T])), f(x_next))
    x = x_next
    if m * r < 1e-6:
        break
    r *= rho   
    num_iter += 1

In [559]:
print(x)

[ 7.99999963e-01  7.99999981e-01  4.45000080e-01  7.99999988e-01
  9.29863342e-09 -4.24999959e-01  1.60000024e-01  2.94999924e-01
 -1.99999984e-01  1.69999995e-01  4.04398846e-09  4.09035204e-09
  4.10072805e-09  4.07762356e-09  4.08899344e-09  3.50083677e-01
  7.75083635e-01  6.15083612e-01  3.20083688e-01  5.20083671e-01
  3.50083677e-01]


In [535]:
c = np.zeros(4*T + 1)
c[:T] = np.array([1.5, 1.4, 1.8, 1.2, 2.6])
c = matrix(c)
H = matrix(H)
g = matrix(g)
A_mtx = matrix(A_mtx)
b = matrix(b)
sol=solvers.lp(c,H,g, A_mtx, b)

     pcost       dcost       gap    pres   dres   k/t
 0:  3.7576e+00 -2.4640e+01  1e+02  2e+00  1e+00  1e+00
 1:  3.6799e+00  8.4040e-03  9e+00  4e-01  2e-01  2e+00
 2:  4.0564e+00  3.5811e+00  9e-01  6e-02  3e-02  3e-01
 3:  4.0650e+00  3.9729e+00  2e-01  1e-02  5e-03  5e-02
 4:  4.0796e+00  4.0552e+00  4e-02  2e-03  1e-03  5e-03
 5:  4.0810e+00  4.0807e+00  4e-04  3e-05  1e-05  5e-05
 6:  4.0810e+00  4.0810e+00  4e-06  3e-07  1e-07  5e-07
 7:  4.0810e+00  4.0810e+00  4e-08  3e-09  1e-09  5e-09
Optimal solution found.


In [560]:
print(sol["x"])


[ 8.00e-01]
[ 8.00e-01]
[ 4.45e-01]
[ 8.00e-01]
[ 6.20e-10]
[-4.25e-01]
[ 1.60e-01]
[ 2.95e-01]
[-2.00e-01]
[ 1.70e-01]
[-7.96e-10]
[-4.45e-10]
[-5.43e-10]
[-7.72e-10]
[-6.87e-10]
[ 3.51e-01]
[ 7.76e-01]
[ 6.16e-01]
[ 3.21e-01]
[ 5.21e-01]
[ 3.51e-01]

